In [ ]:
#@title Day 1-1: Trebuchet?!
from google.colab import files
with open('2023-1-1.txt','r') as f:
  print(sum(map(lambda d: int(d[0]+d[-1]),
   [[c for c in l if '/' < c < ':'] for l in f.read().splitlines()])))

54573


In [7]:
#@title Day 1-2: Trebuchet?!
import re

numbers = dict(
    {n: i for (i,n) in enumerate('zero,one,two,three,four,five,six,seven,eight,nine'.split(','))},
    **{str(i): i for i in range(10)})

from google.colab import files
with open('2023-01-01.txt','r') as f:
  ls = f.read().splitlines()

first = lambda l: numbers[re.search('|'.join(['('+n+')' for n in dict.keys(numbers)]), l).group()]
last = lambda l: numbers[re.search('|'.join(['('+n[::-1]+')' for n in dict.keys(numbers)]), l[::-1]).group()[::-1]]
ns = [10*first(l)+last(l) for l in ls]
print(sum(ns))

54591
